In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,47905
2,Huelva,Confirmados PDIA 14 días,977
3,Huelva,Tasa PDIA 14 días,"190,38525245045503"
4,Huelva,Confirmados PDIA 7 días,487
5,Huelva,Tasa PDIA 7 dias,"94,90032542822067"
6,Huelva,Total Confirmados,48111
7,Huelva,Curados,45907
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  47905.0


/tmp/ipykernel_3900/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13315.0


/tmp/ipykernel_3900/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3900/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3900/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3900/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 827 personas en los últimos 7 días 

Un positivo PCR cada 410 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,47905.0,487.0,977.0,513170.0,94.900325,190.385252,180.0
Huelva-Costa,28308.0,274.0,553.0,289548.0,94.630251,190.987332,95.0
Condado-Campiña,14950.0,168.0,338.0,156231.0,107.533076,216.346308,65.0
Huelva (capital),13315.0,174.0,351.0,143837.0,120.970265,244.026224,62.0
Sierra de Huelva-Andévalo Central,4219.0,45.0,86.0,67391.0,66.774495,127.613480,20.0
Punta Umbría,1326.0,35.0,43.0,15355.0,227.938782,280.039075,14.0
Bollullos Par del Condado,1387.0,33.0,54.0,14387.0,229.373740,375.338848,11.0
Moguer,2132.0,22.0,60.0,21867.0,100.608222,274.386061,11.0
Aljaraque,1712.0,23.0,52.0,21474.0,107.106268,242.153302,9.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),17.0,3.0,10.0,243.0,1234.567901,4115.226337,0.0
Paterna del Campo,270.0,32.0,52.0,3457.0,925.658085,1504.194388,6.0
Corteconcepción,49.0,2.0,3.0,536.0,373.134328,559.701493,2.0
Palos de la Frontera,1187.0,12.0,45.0,11742.0,102.197241,383.239653,5.0
Bollullos Par del Condado,1387.0,33.0,54.0,14387.0,229.373740,375.338848,11.0
Alosno,249.0,1.0,13.0,3933.0,25.425884,330.536486,0.0
Cartaya,2271.0,22.0,63.0,20083.0,109.545387,313.698153,7.0
Higuera de la Sierra,74.0,3.0,4.0,1291.0,232.378002,309.837335,0.0
Palma del Condado (La),1486.0,13.0,31.0,10801.0,120.359226,287.010462,7.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,249.0,1.0,13.0,3933.0,25.425884,330.536486,0.0,0.076923
Palos de la Frontera,1187.0,12.0,45.0,11742.0,102.197241,383.239653,5.0,0.266667
Granada de Río-Tinto (La),17.0,3.0,10.0,243.0,1234.567901,4115.226337,0.0,0.300000
Trigueros,558.0,3.0,9.0,7862.0,38.158229,114.474688,0.0,0.333333
Cumbres Mayores,138.0,1.0,3.0,1749.0,57.175529,171.526587,1.0,0.333333
Cartaya,2271.0,22.0,63.0,20083.0,109.545387,313.698153,7.0,0.349206
Moguer,2132.0,22.0,60.0,21867.0,100.608222,274.386061,11.0,0.366667
Palma del Condado (La),1486.0,13.0,31.0,10801.0,120.359226,287.010462,7.0,0.419355
Aljaraque,1712.0,23.0,52.0,21474.0,107.106268,242.153302,9.0,0.442308
